In [33]:
import requests
import pandas as pd

In [34]:
site_id = 'MLM'
cat_id = 'MLM1000'
num_items = 1000

In [35]:
#sites = requests.get(f'https://api.mercadolibre.com/sites/')
#sites.json()

In [36]:
#cats = requests.get(f'https://api.mercadolibre.com/sites/{site_id}/categories')
#cats.json()

In [37]:
# request and concat num_products products from cat_id and site_id and concat data
data = []
for i in range(0,num_items//50):
    offset = i*50
    url = f'https://api.mercadolibre.com/sites/{site_id}/search?category={cat_id}&offset={offset}'
    request = requests.get(url)
    items = request.json()

    for item in items["results"]:
        item_data = {
            "title": item["title"],
            "catalog_product_id": item["catalog_product_id"],
            "site_id": item["site_id"],
            "category_id": item["category_id"],
            "domain_id": item["domain_id"],
            "thumbnail": item["thumbnail"],
        }

        # Extracting brand, package length, and package weight if available
        for attribute in item.get("attributes", []):
            if attribute["id"] == "BRAND":
                item_data["brand"] = attribute["value_name"]
            elif attribute["id"] == "PACKAGE_LENGTH":
                item_data["package_length"] = attribute["value_name"]
            elif attribute["id"] == "PACKAGE_WEIGHT":
                item_data["package_weight"] = attribute["value_name"]

        data.append(item_data)

df = pd.DataFrame(data)

display(df)

,title,catalog_product_id,site_id,category_id,domain_id,thumbnail,brand,package_length,package_weight
0,Huawei Freebuds Pro 3 Plata Glacial,MLM28292846,MLM,MLM7533,MLM-HEADPHONES,http://http2.mlstatic.com/D_868758-MLA74006755...,Huawei,10.2 cm,200 g
1,Audífonos Inalámbricos Bluetooth Función Carga...,MLM26195158,MLM,MLM7533,MLM-HEADPHONES,http://http2.mlstatic.com/D_707983-MLA71875543...,11,7.6 cm,60 g
2,Bocina Portátil Jbl Flip 6 Bluetooth Aprueba D...,MLM18930466,MLM,MLM59800,MLM-SPEAKERS,http://http2.mlstatic.com/D_804940-MLU74071985...,JBL,10.8 cm,940 g
3,Smart Tv Portátil Hisense H4f Series 40h4030f ...,MLM15254609,MLM,MLM1002,MLM-TELEVISIONS,http://http2.mlstatic.com/D_969661-MLA44217501...,Hisense,99.6 cm,7640 g
4,Asistente Virtual Amazon Echo Dot 5ta Generaci...,MLM19795010,MLM,MLM187752,MLM-SMART_SPEAKERS,http://http2.mlstatic.com/D_976672-MLU72754995...,Amazon,15.4 cm,560 g
...,...,...,...,...,...,...,...,...,...
994,Audífonos Inalámbrico Diadema Con Microfono Bl...,MLM28901848,MLM,MLM7533,MLM-HEADPHONES,http://http2.mlstatic.com/D_832547-CBT71017736...,Vulata,NaN,NaN
995,Soporte Kuiiyer Ku106 De Pared Para Tv/monitor...,MLM28034435,MLM,MLM10574,MLM-TV_AND_MONITOR_MOUNTS,http://http2.mlstatic.com/D_769113-MLA72762515...,Kuiiyer,27.2 cm,1960 g
996,Barra Led Ultravioleta Metalica 18x5 Watts Ste...,MLM22851941,MLM,MLM44588,MLM-LED_STAGE_LIGHTS,http://http2.mlstatic.com/D_863154-MLU73587671...,Steelpro,104.8 cm,1260 g
997,Pila Aa Recargable Duracell Pack 12 Baterías 2...,MLM25887764,MLM,MLM7552,MLM-CELL_BATTERIES,http://http2.mlstatic.com/D_655750-MLU72627744...,Duracell,19.6 cm,380 g


In [38]:
# fill na's in catalog_product_id with the hash of the title to have a distinct id for each product
df['catalog_product_id'] = df['catalog_product_id'].fillna(df['title'].apply(hash))
df['semantic_string'] = df['title'] + ' ' + df['brand'] + ' ' + df['package_length'].fillna('') + ' ' + df['package_weight'].fillna('')
df.isna().sum()

title                   0
catalog_product_id      0
site_id                 0
category_id             0
domain_id               0
thumbnail               0
brand                   0
package_length        423
package_weight        423
semantic_string         0
dtype: int64

In [39]:
df.to_csv(f'./products/{cat_id}_{num_items}.csv')